In [1]:
import numpy as np
import pandas as pd
# import tensorflow
from nltk.corpus import stopwords
import re
import string
from nltk.tokenize import word_tokenize,sent_tokenize
from sklearn.model_selection import train_test_split
# import matplotlib.pyplot as plt
import time
from collections import Counter
from pipe import transform_text_func,FeatureExtractor, ImputeNA, CategoricalEncoding,text
from scipy.sparse import hstack


from sklearn.metrics import accuracy_score, make_scorer,mean_squared_error
from sklearn.preprocessing import StandardScaler
from nltk.stem import PorterStemmer
from scipy.sparse import csr_matrix
import logging
from sklearn.pipeline import make_pipeline, make_union 
#from keras.layers import Input, Dropout, Dense, concatenate, GRU, Embedding, Flatten, Activation, BatchNormalization, PReLU
#from keras.initializers import he_uniform
#from keras.layers import Conv1D
#from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D
#from keras.optimizers import Adam, SGD
#from keras.models import Model
#import gc
# from gensim import corpora,models,similarities
# import gensim

In [44]:
import os
os.listdir()

['tuning.py',
 '.ipynb_checkpoints',
 'training_data.csv',
 'tfidf.ipynb',
 'bow_randomforest.ipynb',
 'pipe.py',
 'Maybe Success!!!!.ipynb',
 'lda.py',
 'linear_one.ipynb',
 '6k.ipynb',
 'word2vec.py',
 'bar.py',
 'failure.ipynb',
 'ridge_test_res.pkl',
 '__pycache__',
 'bow.py',
 'yelp.py',
 'fastText.ipynb',
 'Untitled1.ipynb',
 'dataset',
 'tuning.ipynb',
 'README.md']

In [2]:
train = pd.read_csv('training_data.csv',header= 0 ,delimiter='\t|\n')

/home/kevindong1994/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [3]:
print('-------generate interaction feature between cate-------')
s = time.time()
a = ['age_cat','sex','stay_cat','lang','er','category']

for i,ai in enumerate(a):
    for j,bj in enumerate(a):
        if i<j:
            x = train[ai]
            y = train[bj]
            t = []
            for l in range(train.shape[0]):
                t.append(str(x[l])+' '+ str(y[l]))
            train[ai+'_'+bj] = t
print('time elapsed: ', time.time()-s)

-------generate interaction feature between cate-------
time elapsed:  39.21766400337219


In [4]:
x_total = list(train.comment)
y_total = list(train.score)

In [5]:
print('-------calculate uppercase prob to features-------')
s = time.time()
uppercase = []
for i in x_total:
    length = len(i.split())
    tmp = []
    for j in i:
        if j.isupper():
            tmp.append(j)
    uppercase.append(len(tmp)/length)
uppercase = np.array(uppercase).reshape(-1,1)
scaler = StandardScaler()
new_up = scaler.fit_transform(uppercase)
print('time elapsed: ', time.time()-s)

print('-------choose specific stop words-------')
s=time.time()
x_lower = [sublist.lower() for sublist in x_total]
# punctuation = string.punctuation.replace('!','').replace('?','').replace("'",'').replace('~','')
# regex = re.compile('[' +re.escape(punctuation) +']')
def tokenize(text):
#     text = regex.sub(" ", text) # remove punctuation
    text = text.replace('w/','with ')
#     text = text.replace('&',' and ')
#     text = text.replace('/',' or ')
    text = text.replace('.',' ')
    text = text.split()
    return(text)
x_lower = [tokenize(i) for i in x_lower]
x_unlist = []
for i in x_lower:
    x_unlist += i
vocab_dic = Counter(x_unlist)
stopwords_num = 250
# print([i[0] for i in vocab_dic.most_common(stopwords_num)],'\n\n')
print('total vocab: ',len(vocab_dic.most_common()))
maxfeature = len([i[0] for i in vocab_dic.most_common() if i[1]>1])
print('vocab size frequency > 1: ', maxfeature)

stop = [i[0] for i in vocab_dic.most_common(stopwords_num)]
from itertools import compress
x_5level = [list(compress(x_lower, list(np.array(y_total)==i))) for i in np.unique(y_total)]

x_5level_unlist = [[],[],[],[],[],[],[],[],[],[],[]]
for i in range(len(x_5level)):
    for j in x_5level[i]:
        x_5level_unlist[i] += j
multilevel_vocab = []
for i in range(11):
    multilevel_vocab.append(Counter(x_5level_unlist[i]))


from collections import defaultdict
multilevel_stop = defaultdict(list)
for i in range(11):
    tt = len(x_5level_unlist[i])
    for j in stop:
        multilevel_stop[j].append(multilevel_vocab[i][j]/tt)

stop_var = [(key,np.std(value)*1000) for key,value in multilevel_stop.items() ]
stop = [i[0] for i in stop_var if i[1]<0.5]

print('time elapsed: ', time.time()-s)



def tokenize(text):
    try:
        punctuation = string.punctuation.replace('!','').replace('?','').replace("'",'').replace("/",'').replace("@",'').replace('"','')
        regex = re.compile('[' +re.escape(punctuation) +']')
        text = regex.sub(" ", text) # remove punctuation
        text = text.replace('w/',' with ')
        text = text.replace('&',' and ')
        text = text.replace("/",' or ')
        text = text.replace('""',' sarcasm ')
        text = text.replace("'d",' ')
        text = text.replace("'s",' ')
        text = text.replace("'re",' ')
        text = text.replace("'ll",' ')
        text = text.replace("'ve",' ')    
#         text = text.replace('.0','')
        text = text.replace('.',' ')

        ps = PorterStemmer()
        tokens = []
        tokens_ = [s.split() for s in sent_tokenize(text)]
        for token_by_sent in tokens_:
            tokens += token_by_sent   
        filtered_tokens = [ps.stem(w.lower()) for w in tokens]
        return filtered_tokens
    except TypeError as e: print(text,e)


-------calculate uppercase prob to features-------
time elapsed:  1.3343238830566406
-------choose specific stop words-------
total vocab:  44702
vocab size frequency > 1:  20256
time elapsed:  1.6845476627349854


In [6]:
print('-------transform to features-------')
s=time.time()
onehot_list = ['age_cat', 'sex', 'stay_cat', 'lang', 'er','age_cat_sex', 'age_cat_stay_cat',
       'age_cat_lang', 'age_cat_er', 'age_cat_category', 'sex_stay_cat',
       'sex_lang', 'sex_er', 'sex_category', 'stay_cat_lang', 'stay_cat_er',
       'stay_cat_category', 'lang_er', 'lang_category', 'er_category']
onehot_pipeline = make_pipeline(FeatureExtractor(onehot_list),
                                CategoricalEncoding('OneHot'),
                                )
descrip_pipeline = make_pipeline(FeatureExtractor('comment'),
                                text(method='tfidf', ngram = 3, max_f = maxfeature, 
                                     binary = True, stopwords=stop,tokenizer=tokenize,analyzer ='word'))

feature_union = make_union(
    onehot_pipeline,
    descrip_pipeline
)
X = feature_union.fit_transform(train)
print('time elapsed: ', time.time()-s)

print('-------add length and upper prob to features-------')
s = time.time()
length = np.array([(X[i,]!=0).sum() for i in range(X.shape[0])]).reshape(-1,1)
scaler = StandardScaler()
new_l = scaler.fit_transform(length)
X = hstack([X,new_l],format='csr')
X = hstack([X,new_up],format='csr')

print('X shape: ',X.shape)
print('time elapsed: ', time.time()-s)

-------transform to features-------
time elapsed:  61.7270393371582
-------add length and upper prob to features-------


/home/kevindong1994/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


X shape:  (151824, 20702)
time elapsed:  30.358232736587524


In [7]:
X.shape,len(y_total)

((151824, 20702), 151824)

In [8]:
import numpy as np
def int2vector(i):
    return np.array([1]*(i+1)+[0]*(10-i)).reshape(11,1)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_total, test_size=0.33, random_state=42)

y_train = [int2vector(i) for i in y_train]

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
def to_predict(row):
        for i in range(11):
            if row[-1]>0.5:
                return 10
            elif row[i]<0.5:
                return i-1

logging = {"i":0,"epoch":0,"epochs":50}            
            
def generate_batch(features,labels,ids):
    global logging
    if logging["i"]==len(ids)-1:
        inner_i = logging["i"]
        logging["i"] = 0
        logging["epoch"] += 1
        return features[ids[inner_i]:],labels[ids[inner_i]:]
    else:
        inner_i = logging["i"]
        logging["i"] += 1
        return features[ids[inner_i]:ids[logging["i"]]],labels[ids[inner_i]:ids[logging["i"]]]



def sum_to_pred(row):
    r = np.rint(np.sum(row))
    if r>10:
        return 10
    elif r<0:
        return 0
    else:
        return r

train_data = [X_train[i,] for i in range(X_train.shape[0])]
#train_data = [np.array(each.todense())[0] for each in train_data]
test_data = X_test
test_data_features = [X_test[i,] for i in range(X_test.shape[0])]
#test_data_features = [np.array(each.todense())[0] for each in test_data_features]

batch_size = 50
num_features=X_train.shape[1]
H = 5

logging["i"] = 0
logging["epoch"] = 0
ids = list(range(0,X_train.shape[0],batch_size))
print("feature:%i,H:%i,batch:%i"%(num_features,H,batch_size))


graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32,shape=[None,num_features])
    y_ = tf.placeholder(tf.float32,shape=[None,11,1])
    new_y = tf.squeeze(y_)
    W_1 = tf.Variable(tf.random_uniform([num_features,H]))
    b_1 = tf.Variable(tf.zeros([H]))

    W_2 = tf.Variable(tf.random_uniform([H,11]))
    b_2 = tf.Variable(tf.zeros([11]))

    temp_1 = tf.nn.relu(tf.matmul(x,W_1,a_is_sparse=True) + b_1)
    temp_2 = tf.matmul(temp_1,W_2) + b_2
    output = tf.nn.sigmoid(temp_2)
    #loss = tf.reduce_mean(tf.square(tf.subtract(output,new_y)))
    loss = tf.losses.mean_squared_error(output,new_y)
    opt = tf.train.AdamOptimizer(learning_rate=0.01)
    opt_op = opt.minimize(loss)
    init = tf.global_variables_initializer()

# with tf.Session(graph = graph) as session:

sess = tf.InteractiveSession(graph=graph)
sess.run(init)
print("Initialized")
step = 0 
while logging["epoch"]<logging["epochs"]:
    step+=1
    batch_x,batch_y = generate_batch(train_data,y_train,ids)
    batch_x = [np.array(each.todense())[0] for each in batch_x]
    feed_dict={x:batch_x,y_:batch_y}
    _,loss_val = sess.run([opt_op,loss],feed_dict=feed_dict)
    if step % 2000 == 0:
        print("Step: %i, Epoch: %i, Loss:%f"%(step,logging["epoch"],loss_val))



/home/kevindong1994/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


feature:20702,H:5,batch:50
Initialized
Step: 2000, Epoch: 0, Loss:0.087136
Step: 4000, Epoch: 1, Loss:0.079032
Step: 6000, Epoch: 2, Loss:0.093925
Step: 8000, Epoch: 3, Loss:0.081407
Step: 10000, Epoch: 4, Loss:0.058652
Step: 12000, Epoch: 5, Loss:0.060201
Step: 14000, Epoch: 6, Loss:0.055470
Step: 16000, Epoch: 7, Loss:0.047501
Step: 18000, Epoch: 8, Loss:0.060264
Step: 20000, Epoch: 9, Loss:0.054659
Step: 22000, Epoch: 10, Loss:0.046703
Step: 24000, Epoch: 11, Loss:0.047577
Step: 26000, Epoch: 12, Loss:0.049691
Step: 28000, Epoch: 13, Loss:0.044234
Step: 30000, Epoch: 14, Loss:0.039882
Step: 32000, Epoch: 15, Loss:0.040288
Step: 34000, Epoch: 16, Loss:0.039945
Step: 36000, Epoch: 17, Loss:0.046995
Step: 38000, Epoch: 18, Loss:0.036522
Step: 40000, Epoch: 19, Loss:0.040401
Step: 42000, Epoch: 20, Loss:0.039139
Step: 44000, Epoch: 21, Loss:0.039325
Step: 46000, Epoch: 22, Loss:0.053046
Step: 48000, Epoch: 23, Loss:0.038395
Step: 50000, Epoch: 24, Loss:0.039745
Step: 52000, Epoch: 25, L

In [10]:
test_data_features = [np.array(each.todense())[0] for each in test_data_features]
result = sess.run(output,feed_dict={x:test_data_features})
print("Calculated Finished")
#result = sess.run(output,feed_dict={x:test_data_features})
result_label = [to_predict(each) for each in result]
from sklearn.metrics import mean_squared_error
sd = mean_squared_error(result_label, y_test)
print(sd)


Calculated Finished
5.266436469602012


In [15]:
batch_x[:3]

[<1x20702 sparse matrix of type '<class 'numpy.float64'>'
 	with 23 stored elements in Compressed Sparse Row format>,
 <1x20702 sparse matrix of type '<class 'numpy.float64'>'
 	with 48 stored elements in Compressed Sparse Row format>,
 <1x20702 sparse matrix of type '<class 'numpy.float64'>'
 	with 25 stored elements in Compressed Sparse Row format>]

In [11]:
import mord

In [40]:
classifier = mord.OrdinalRidge(max_iter=100000,solver="sag",tol=0.0001)


In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y_total, test_size=0.33, random_state=42)

In [41]:
classifier.fit(X_train,y_train)

OrdinalRidge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=100000,
       normalize=False, random_state=None, solver='sag', tol=0.0001)

In [42]:
prediction = classifier.predict(X_test)

In [43]:
mean_squared_error(prediction, y_test)

3.4155123547962156

In [17]:
graph = tf.Graph()
with graph.as_default():
    x = tf.sparse_placeholder(tf.float32,shape=[None,1,num_features])
    y_ = tf.placeholder(tf.float32,shape=[None,11,1])
    new_y = tf.squeeze(y_)
    W_1 = tf.Variable(tf.random_uniform([num_features,H]))
    b_1 = tf.Variable(tf.zeros([H]))

    W_2 = tf.Variable(tf.random_uniform([H,11]))
    b_2 = tf.Variable(tf.zeros([11]))

    temp_1 = tf.nn.relu(tf.matmul(tf.squeeze(tf.sparse_tensor_to_dense(x)),W_1,a_is_sparse=True) + b_1)
    temp_2 = tf.matmul(temp_1,W_2) + b_2
    output = tf.nn.sigmoid(temp_2)
    #loss = tf.reduce_mean(tf.square(tf.subtract(output,new_y)))
    loss = tf.losses.mean_squared_error(output,new_y)
    opt = tf.train.AdamOptimizer(learning_rate=0.01)
    opt_op = opt.minimize(loss)
    init = tf.global_variables_initializer()

# with tf.Session(graph = graph) as session:

sess = tf.InteractiveSession(graph=graph)
sess.run(init)
print("Initialized")
step = 0 
while logging["epoch"]<logging["epochs"]:
    step+=1
    batch_x,batch_y = generate_batch(train_data,y_train,ids)
    feed_dict={x:batch_x,y_:batch_y}
    _,loss_val = sess.run([opt_op,loss],feed_dict=feed_dict)
    if step % 2000 == 0:
        print("Step: %i, Epoch: %i, Loss:%f"%(step,logging["epoch"],loss_val))



Initialized


ValueError: setting an array element with a sequence.

In [31]:
graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32,shape=[None,num_features])
    y_ = tf.placeholder(tf.float32,shape=[None,11,1])
    new_y = tf.squeeze(y_)
    W_1 = tf.Variable(tf.random_uniform([num_features,H]))
    b_1 = tf.Variable(tf.zeros([H]))

    W_2 = tf.Variable(tf.random_uniform([H,11]))
    b_2 = tf.Variable(tf.zeros([11]))

    temp_1 = tf.nn.relu(tf.matmul(x,W_1) + b_1)
    temp_2 = tf.matmul(temp_1,W_2) + b_2
    output = tf.nn.sigmoid(temp_2)
    #loss = tf.reduce_mean(tf.square(tf.subtract(output,new_y)))
    loss = tf.losses.mean_squared_error(output,new_y)
    opt = tf.train.AdamOptimizer(learning_rate=0.01)
    opt_op = opt.minimize(loss)
    init = tf.global_variables_initializer()

# with tf.Session(graph = graph) as session:

batch_x,batch_y = generate_batch(train_data,y_train,ids)
batch_x = [np.array(each.todense())[0] for each in batch_x]
feed_={x:batch_x,y_:batch_y}

sess = tf.InteractiveSession(graph=graph)
sess.run(init)
sess.run(x,feed_dict=feed_)

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.8488746 ,  0.5922257 ],
       [ 0.        ,  0.        ,  1.        , ...,  0.        ,
        -0.37600356,  2.0009363 ],
       [ 0.        ,  1.        ,  0.        , ...,  0.        ,
         0.99006844,  0.07322705],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.48108602,  0.24004802],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.06075616, -0.34691468],
       [ 0.        ,  0.        ,  1.        , ...,  0.        ,
         0.3595737 , -0.46430722]], dtype=float32)

{<tensorflow.python.framework.sparse_tensor.SparseTensor at 0x7f128ead45f8>: [<1x20702 sparse matrix of type '<class 'numpy.float64'>'
  	with 28 stored elements in Compressed Sparse Row format>,
  <1x20702 sparse matrix of type '<class 'numpy.float64'>'
  	with 36 stored elements in Compressed Sparse Row format>,
  <1x20702 sparse matrix of type '<class 'numpy.float64'>'
  	with 37 stored elements in Compressed Sparse Row format>,
  <1x20702 sparse matrix of type '<class 'numpy.float64'>'
  	with 38 stored elements in Compressed Sparse Row format>,
  <1x20702 sparse matrix of type '<class 'numpy.float64'>'
  	with 28 stored elements in Compressed Sparse Row format>,
  <1x20702 sparse matrix of type '<class 'numpy.float64'>'
  	with 25 stored elements in Compressed Sparse Row format>,
  <1x20702 sparse matrix of type '<class 'numpy.float64'>'
  	with 52 stored elements in Compressed Sparse Row format>,
  <1x20702 sparse matrix of type '<class 'numpy.float64'>'
  	with 35 stored element

In [ ]:
train_data[:3]

In [ ]:
test_data_features[:3]

In [35]:
test_data_features = [np.array(each.todense())[0] for each in test_data_features]
result = sess.run(output,feed_dict={x:test_data_features})

In [37]:
#result = sess.run(output,feed_dict={x:test_data_features})
result_label = [to_predict(each) for each in result]
from sklearn.metrics import mean_squared_error
sd = mean_squared_error(result_label, y_test)
print(sd)


8.90679014809788


In [ ]:

result = sess.run(output,feed_dict={x:test_data_features})
result_label = [sum_to_pred(each) for each in result]
from sklearn.metrics import mean_squared_error
custom = mean_squared_error(result_label, y_test)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
def to_predict(row):
        for i in range(11):
            if row[-1]>0.5:
                return 10
            elif row[i]<0.5:
                return i-1

logging = {"i":0,"epoch":0,"epochs":20}            
            
def generate_batch(features,labels,ids):
    global logging
    if logging["i"]==len(ids)-1:
        inner_i = logging["i"]
        logging["i"] = 0
        logging["epoch"] += 1
        return features[ids[inner_i]:],labels[ids[inner_i]:]
    else:
        inner_i = logging["i"]
        logging["i"] += 1
        return features[ids[inner_i]:ids[logging["i"]]],labels[ids[inner_i]:ids[logging["i"]]]



def sum_to_pred(row):
    r = np.rint(np.sum(row))
    if r>10:
        return 10
    elif r<0:
        return 0
    else:
        return r

train_data = [X_train[i,] for i in range(X_train.shape[0])]
train_data = [np.array(each.todense())[0] for each in train_data]
test_data = X_test
test_data_features = [X_test[i,] for i in range(X_test.shape[0])]
test_data_features = [np.array(each.todense())[0] for each in test_data_features]
    
def tuning(H,batch_size,num_features=X_train.shape[1]):   
    global logging
    logging["i"] = 0
    logging["epoch"] = 0
    ids = list(range(0,X_train.shape[0],batch_size))
    print("feature:%i,H:%i,batch:%i"%(num_features,H,batch_size))
    #num_features = 5000

    #batch_size = 10

    #H = 5
    graph = tf.Graph()
    with graph.as_default():
        x = tf.placeholder(tf.float32,shape=[None,num_features])
        y_ = tf.placeholder(tf.float32,shape=[None,11,1])
        new_y = tf.squeeze(y_)
        W_1 = tf.Variable(tf.random_uniform([num_features,H]))
        b_1 = tf.Variable(tf.zeros([H]))

        W_2 = tf.Variable(tf.random_uniform([H,11]))
        b_2 = tf.Variable(tf.zeros([11]))

        temp_1 = tf.nn.relu(tf.matmul(x,W_1) + b_1)
        temp_2 = tf.matmul(temp_1,W_2) + b_2
        output = tf.nn.sigmoid(temp_2)
        #loss = tf.reduce_mean(tf.square(tf.subtract(output,new_y)))
        loss = tf.losses.mean_squared_error(output,new_y)
        opt = tf.train.AdamOptimizer(learning_rate=0.01)
        opt_op = opt.minimize(loss)
        init = tf.global_variables_initializer()

    # with tf.Session(graph = graph) as session:

    sess = tf.InteractiveSession(graph=graph)
    sess.run(init)
    print("Initialized")
    step = 0 
    while logging["epoch"]<logging["epochs"]:
        step+=1
        batch_x,batch_y = generate_batch(train_data,y_train,ids)
        feed_dict={x:batch_x,y_:batch_y}
        _,loss_val = sess.run([opt_op,loss],feed_dict=feed_dict)
        if step % 2000 == 0:
            print("Step: %i, Epoch: %i, Loss:%f"%(step,logging["epoch"],loss_val))

    result = sess.run(output,feed_dict={x:test_data_features})
    result_label = [to_predict(each) for each in result]
    from sklearn.metrics import mean_squared_error
    sd = mean_squared_error(result_label, y_test)


    result = sess.run(output,feed_dict={x:test_data_features})
    result_label = [sum_to_pred(each) for each in result]
    from sklearn.metrics import mean_squared_error
    custom = mean_squared_error(result_label, y_test)
    sess.close()
    return sd,custom

In [ ]:
print(tuning(5,50))